<a href="https://colab.research.google.com/github/Paradorn657/help/blob/master/Saving_and_Loading_Models_Do%20it%20yourself.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
%tensorflow_version 2.x
from tensorflow.keras import layers

In [4]:
splits,info = tfds.load("cats_vs_dogs",with_info=True,as_supervised=True,split=['train[:80%]', 'train[80%:]'])
(train_examples,validation_examples) = splits

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteF3JDGI/cats_vs_dogs-train.tfrecord
Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [0]:
def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return  image, label

num_examples = info.splits['train'].num_examples

BATCH_SIZE = 32
IMAGE_RES = 224

train_batches      = train_examples.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.cache().map(format_image).batch(BATCH_SIZE).prefetch(1)

บรรทัด 2 สุดท้าย คือการเอา dataset มากำหนด batch_size และ สลับเพื่อไม่ให้ nearal จดจำลำดับการฟีดข้อมูล train exam เก็บข้อมูลเป็นคู่ๆ 


In [0]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,input_shape=(224, 224,3))

In [0]:
feature_extractor.trainable = False

In [8]:
model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(2)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [9]:
model.compile(
  optimizer='adam', 
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

EPOCHS = 1
history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)


582/582 [==============================] - 421s 723ms/step - loss: 0.0507 - accuracy: 0.9830 - val_loss: 0.0335 - val_accuracy: 0.9890


In [12]:
class_names = np.array(info.features['label'].names)
class_names

array(['cat', 'dog'], dtype='<U3')

In [27]:
image_batch, label_batch = next(iter(train_batches.take(1)))
image_batch = image_batch.numpy()
label_batch = label_batch.numpy()

predicted_batch = model.predict(image_batch)
predicted_batch = tf.squeeze(predicted_batch).numpy()
predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = class_names[predicted_ids]
predicted_class_names

array(['cat', 'dog', 'cat', 'cat', 'cat', 'dog', 'dog', 'cat', 'dog',
       'cat', 'dog', 'cat', 'cat', 'dog', 'dog', 'cat', 'cat', 'cat',
       'dog', 'dog', 'cat', 'cat', 'dog', 'dog', 'cat', 'cat', 'cat',
       'dog', 'cat', 'cat', 'dog', 'cat'], dtype='<U3')

บรรทัดแรกคือ การเอาข้อมูล bacth แรก ออกมา โดยมีทั้งรูป และ label next คือเอาก็ข้อมูลถัดไป แล้วแปลงเป็น numpy arr แล้วให้ model predict ค่า label ออกมา โดยหาตำแหน่งที่มีค่ามาก 0 คือ cat 1 คือ dog

In [25]:
predicted_ids


array([1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1])

In [26]:
label_batch

array([1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1])

# Part 3: Save as Keras .h5 

In [28]:
export_path_keras = "./{}.h5".format("cat_dog")
print(export_path_keras)

model.save(export_path_keras)


./cat_dog.h5


export_path_sm = "./{}".format("cat_dog")

print(export_path_sm)

tf.saved_model.save(model, export_path_sm)
# แบบนี้จะเซฟในรูปแบบ tensorflow 
----------------------------------------------

## Part 4: Load the Keras .h5 Model
# `custom_objects` tells keras how to load a `hub.KerasLayer`
# แบบนี้โหลดแบบ tensorflow
reloaded_sm = tf.saved_model.load(export_path_sm)

----------------------------------------------

In [29]:
reloaded_model = tf.keras.models.load_model(
    export_path_keras,
    custom_objects={'KerasLayer': hub.KerasLayer}
    )

ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [30]:
reloaded_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [35]:
reload_predict = reloaded_model.predict(image_batch)
reload_label_ids = np.argmax(reload_predict, axis=-1)
reload_label_ids

array([0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0])

In [36]:
label_batch

array([0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0])

In [37]:
accurate_reload_label = abs(reload_label_ids-label_batch)
accurate_reload_label

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# ผิดอันที่สาม อยู่อันเดียว

In [39]:
!zip -r model.zip {export_path_keras}

  adding: cat_dog.h5 (deflated 9%)
